From https://planetarycomputer.microsoft.com/dataset/cil-gdpcir-cc0#Ensemble-example

In [1]:
import planetary_computer
import pystac_client

import xarray as xr
import numpy as np
import pandas as pd
from dask.diagnostics import ProgressBar
from tqdm.auto import tqdm

import os
import re
import glob
import shutil
import zipfile
from pathlib import Path

import difflib
from scipy.spatial import KDTree

import matplotlib.pyplot as plt
import geopandas as gpd
import regionmask
import cartopy.crs as ccrs

from netCDF4 import Dataset

from carbonplan import styles  # noqa: F401
import intake
import cmip6_downscaling


Load and organise data

In [6]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)
collection_cc0 = catalog.get_collection("cil-gdpcir-cc0")
collection_cc0.summaries.to_dict()
scenarios = ["ssp245"]
for scenario in scenarios:
    search = catalog.search(
        collections=["cil-gdpcir-cc0", "cil-gdpcir-cc-by"], # both creative licenses
        query={"cmip6:experiment_id": {"eq": f"{scenario}"}}, # stick with 245 for the moment 
    )
    ensemble = search.item_collection()
    print(len(ensemble))
    
    # read in only the precipitation
    variable_id = "pr"
    
    datasets_by_model = []
    
    for item in tqdm(ensemble):
        asset = item.assets[variable_id]
        datasets_by_model.append(
            xr.open_dataset(asset.href, **asset.extra_fields["xarray:open_kwargs"])
        )
    
    all_datasets = xr.concat(
        datasets_by_model,
        dim=pd.Index([ds.attrs["source_id"] for ds in datasets_by_model], name="model"),
        combine_attrs="drop_conflicts",
    )
    
    #Subset for Malawi and 2025-2100
    
    subset = all_datasets.pr.sel(
        lon=slice(32.67161823,35.91841716),
        lat=slice(-17.12627881, -9.36366167),
        time=slice("2025-01-01", "2071-01-01"),
    )
    
    subset.to_netcdf(f"/Users/rem76/Desktop/Climate_change_health/Data/Precipitation_data/Downscaled_CMIP6_data_CIL/CIL_subsetted_all_model_{scenario}.nc")

23


  0%|          | 0/23 [00:00<?, ?it/s]

PermissionError: [Errno 13] Permission denied: '/Users/rem76/Desktop/Climate_change_health/Data/Precipitation_data/Downscaled_CMIP6_data_CIL/CIL_subsetted_all_model_ssp245.nc'

In [ ]:
    subset.to_netcdf(f"/Users/rem76/Desktop/Climate_change_health/Data/Precipitation_data/Downscaled_CMIP6_data_CIL/CIL_subsetted_all_model_{scenario}.nc")

Find lowest, median, and highest value model across all lat/long and across all time points

In [7]:
subset_xr = xr.Dataset(
    data_vars=dict(
        pr=(["time", "lat","lon", "model"],np.asarray(subset.data)),
    ),
    coords=dict(
        time=subset.time.data,
        lat=subset.lat.data,
        lon=subset.lon.data,
        model=subset.model.data
    ),
    attrs=dict(description="Weather related data."),
)


ClientAuthenticationError: Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:356fa9fc-a01e-0046-0fbb-5da77d000000
Time:2025-01-03T08:44:03.5582125Z
ErrorCode:AuthenticationFailed
authenticationerrordetail:Signature not valid in the specified time frame: Start [Wed, 01 Jan 2025 21:42:40 GMT] - Expiry [Thu, 02 Jan 2025 22:27:40 GMT] - Current [Fri, 03 Jan 2025 08:44:03 GMT]
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthenticationFailed</Code><Message>Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:356fa9fc-a01e-0046-0fbb-5da77d000000
Time:2025-01-03T08:44:03.5582125Z</Message><AuthenticationErrorDetail>Signature not valid in the specified time frame: Start [Wed, 01 Jan 2025 21:42:40 GMT] - Expiry [Thu, 02 Jan 2025 22:27:40 GMT] - Current [Fri, 03 Jan 2025 08:44:03 GMT]</AuthenticationErr

Now find overall lowest/median/highest for model ensemble

# Combine with grids for facilities

In [20]:
ANC = True
Inpatient = False
multiplier = 1 # no need for multiplier 
years = range(2025, 2071) # final date is 1st Jan 2100
month_lengths = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] * len(years)
window_size = 5

if ANC:
    reporting_data = pd.read_csv(
        "/Users/rem76/Desktop/Climate_change_health/Data/monthly_reporting_ANC_by_smaller_facility_lm.csv")
elif Inpatient:
    reporting_data = pd.read_csv(
        "/Users/rem76/Desktop/Climate_change_health/Data/monthly_reporting_Inpatient_by_smaller_facility_lm.csv")
general_facilities = gpd.read_file("/Users/rem76/Desktop/Climate_change_health/Data/facilities_with_districts.shp")

facilities_with_lat_long = pd.read_csv(
    "/Users/rem76/Desktop/Climate_change_health/Data/facilities_with_lat_long_region.csv")

/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/4112377517.py:16: DtypeWarning: Columns (58,59,105,127,136,142,149,150,258,285,296,319,344,345,360,393,394,427,428,437,449,450,452,453,461,462,478,479,489,490,492,493,494,497,498,499,500,501,502,503,572,580,585,586,587,588,591,592,593,594,607,608,609,610,619,620,621,622,626,634,872,887,967,978,1066,1510) have mixed types. Specify dtype option on import or set low_memory=False.
  facilities_with_lat_long = pd.read_csv(


In [21]:
def unzip_all_in_directory(directory):
    """
    Unzips all .zip files in the specified directory, extracting each into a separate folder.

    Parameters:
        directory (str): The path to the folder containing the .zip files.
    """
    for filename in os.listdir(directory):
        if filename.endswith('.zip'):
            file_path = os.path.join(directory, filename)
            extract_dir = os.path.join(directory, filename[:-4])
            os.makedirs(extract_dir, exist_ok=True)

            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_dir)
            except zipfile.BadZipFile:
                print(f"Skipped {filename}: not a valid zip file.")

def get_facility_lat_long(reporting_facility, facilities_df, cutoff=0.90, n_matches=3):
    """
    Function to find the closest matching facility name and return its latitude and longitude.

    Parameters:
    - reporting_facility: The facility name for which latitude and longitude are needed.
    - facilities_df : DataFrame containing facility names ('Fname') and their corresponding latitudes ('A109__Latitude') and longitudes ('A109__Longitude').
    - cutoff: The minimum similarity score for a match. Default is 0.90.
    - n_matches: The maximum number of matches to consider. Default is 3.

    Returns: match_name, lat_for_facility, long_for_facility

    """
    matching_facility_name = difflib.get_close_matches(reporting_facility, facilities_df['Fname'], n=n_matches,
                                                       cutoff=cutoff)

    if matching_facility_name:
        match_name = matching_facility_name[0]  # Access the string directly
        lat_for_facility = facilities_df.loc[facilities_df['Fname'] == match_name, "A109__Latitude"].iloc[0]
        long_for_facility = facilities_df.loc[facilities_df['Fname'] == match_name, "A109__Longitude"].iloc[0]
        return match_name, lat_for_facility, long_for_facility
    else:
        return np.nan, np.nan, np.nan

def extract_nc_files_from_unzipped_folders(directory):
    """
    Searches for .nc files in the specified directory and all its subfolders,
    and copies them to the output directory, maintaining the folder structure.

    Parameters:
        directory (str): The path to the folder containing the unzipped folders.
    """
    output_directory = os.path.join(directory, 'nc_files')
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, _, files in os.walk(directory):
        # Skip the output directory to prevent recursive copying
        if root == output_directory:
            continue

        for filename in files:
            if filename.endswith('.nc'):
                source_file_path = os.path.join(root, filename)
                destination_file_path = os.path.join(output_directory, filename)

                # Only copy if the file does not already exist in the output directory
                if not os.path.exists(destination_file_path):
                    shutil.copy2(source_file_path, output_directory)

In [24]:
base_dir = "/Users/rem76/Desktop/Climate_change_health/Data/Precipitation_data/Downscaled_CMIP6_data_CIL/"
nc_file_directory = os.path.join(base_dir, 'nc_files')
# NB these are daily 
scenarios = ["ssp245", "ssp585"]  # don't have ssp19 scenario

data_by_model_and_grid = {}
for scenario in scenarios:
    print(scenario)
    scenario_directory = os.path.join(base_dir, scenario)

    grid_centroids = {}
    cumulative_sum_by_models = {}
    file_path_downscaled = f"/Users/rem76/Desktop/Climate_change_health/Data/Precipitation_data/Downscaled_CMIP6_data_CIL/CIL_subsetted_all_model_{scenario}.nc"
    data_all_models = xr.open_dataset(file_path_downscaled)
    
    ## Get models of interest - min, med, max
    # Assuming 'pr' is the variable representing precipitation in the dataset
    pr_aggregated = data_all_models.mean(dim=["lat", "lon", "time"])  # Work with the 'pr' DataArray 

    # Find the model with the lowest value
    min_model_object = pr_aggregated['pr'].idxmin(dim="model")
    min_model = min_model_object.values.item()
    
    median_value = pr_aggregated['pr'].median(dim="model").values.item()
    # Find the model with the median value
    sorted_models = pr_aggregated.sortby("model")
    n_models = len(pr_aggregated.model)
    median_index = n_models // 2
    median_model_object = sorted_models["model"][median_index]
    median_model = median_model_object.values.item()
    
    # Find the model with the highest value
    max_model_object = pr_aggregated['pr'].idxmax(dim="model")
    max_model = max_model_object.values.item()

    models_of_interest = [min_model, median_model, max_model]
    print("Models of interest", models_of_interest)
    # see which facilities have reporting data and data on latitude and longitude
    weather_df_lowest_window = pd.DataFrame()
    weather_df_median_window = pd.DataFrame()
    weather_df_highest_window = pd.DataFrame()
    
    weather_df_lowest_monthly = pd.DataFrame()
    weather_df_median_monthly = pd.DataFrame()
    weather_df_highest_monthly = pd.DataFrame()
    for model in models_of_interest:
        data_per_model = data_all_models.sel(model=model)
        pr_data = data_per_model.variables['pr'][:]  # in kg m-2 s-1 = mm s-1 x 86400 to get to day
        lat_data = data_per_model.variables['lat'][:]
        lon_data = data_per_model.variables['lon'][:]
        lon_grid, lat_grid = np.meshgrid(lon_data, lat_data)
        centroids = np.column_stack((lat_grid.ravel(), lon_grid.ravel()))

        # Store centroids
        grid_centroids[model] = centroids
        grid_dictionary = {}
        grid = 0
        for i in lat_data:
            for j in lon_data:
                precip_data_for_grid = data_per_model.sel(lat=i, lon=j, method="nearest")  # across all time points
                grid_dictionary[grid] = precip_data_for_grid.pr.data
                grid += 1
        data_by_model_and_grid[model] = grid_dictionary

    for reporting_facility in reporting_data.columns:
        print(reporting_facility)
        grid_precipitation_for_facility = {}
        match_name, lat_for_facility, long_for_facility = get_facility_lat_long(reporting_facility, facilities_with_lat_long)
        if not np.isnan(long_for_facility) and not np.isnan(lat_for_facility):
            facility_location = np.array([lat_for_facility, long_for_facility])
            kd_trees_by_model = {}
            
            # Loop over each model of interest
            for model in models_of_interest:
                centroids = grid_centroids[model]
                kd_tree = KDTree(centroids)
                distance, closest_grid_index = kd_tree.query(facility_location)
                grid_precipitation_for_facility[model] = data_by_model_and_grid[model][closest_grid_index].data
    
                cumulative_sum_monthly = []
                cumulative_sum_window = []
    
                begin_day = 0
                # Calculate monthly cumulative sums
                for month_idx, month_length in enumerate(month_lengths):
                    days_for_grid_monthly = grid_precipitation_for_facility[model][begin_day:begin_day + month_length]
                    cumulative_sums_monthly = [
                        sum(days_for_grid_monthly)
                    ]
                    max_cumulative_sums_monthly = max(cumulative_sums_monthly)
                    cumulative_sum_monthly.append(max_cumulative_sums_monthly)
                    begin_day += month_length
    
                begin_day = 0
                # Calculate windowed cumulative sums
                for month_idx, month_length in enumerate(month_lengths):
                    days_for_grid_window = grid_precipitation_for_facility[model][begin_day:begin_day + month_length]
    
                    cumulative_sums_window = [
                        sum(days_for_grid_window[day:day + window_size])
                        for day in range(month_length - window_size + 1)
                    ]
    
                    max_cumulative_sums_window = max(cumulative_sums_window)
                    cumulative_sum_window.append(max_cumulative_sums_window)
                    begin_day += month_length
    
                # Assign the calculated data to the correct dataframe based on the model
                if model == min_model:
                    weather_df_lowest_monthly[reporting_facility] = cumulative_sum_monthly
                    weather_df_lowest_window[reporting_facility] = cumulative_sum_window
                elif model == median_model:
                    weather_df_median_monthly[reporting_facility] = cumulative_sum_monthly
                    weather_df_median_window[reporting_facility] = cumulative_sum_window
                elif model == max_model:
                    weather_df_highest_monthly[reporting_facility] = cumulative_sum_monthly
                    weather_df_highest_window[reporting_facility] = cumulative_sum_window
        
    if ANC:
            weather_df_lowest_window.to_csv(Path(scenario_directory) / f"lowest_model_daily_prediction_weather_by_facility_KDBall_ANC_downscaled_CIL_{scenario}.csv", index=False)
            weather_df_median_window.to_csv(Path(scenario_directory) / f"median_model_daily_prediction_weather_by_facility_KDBall_ANC_downscaled_CIL_{scenario}.csv", index=False)
            weather_df_highest_window.to_csv(Path(scenario_directory) / f"highest_model_daily_prediction_weather_by_facility_KDBall_ANC_downscaled_CIL_{scenario}.csv", index=False)
    
            weather_df_lowest_monthly.to_csv(Path(scenario_directory) / f"lowest_model_monthly_prediction_weather_by_facility_KDBall_ANC_downscaled_CIL_{scenario}.csv", index=False)
            weather_df_median_monthly.to_csv(Path(scenario_directory) / f"median_model_monthly_prediction_weather_by_facility_KDBall_ANC_downscaled_CIL_{scenario}.csv", index=False)
            weather_df_highest_monthly.to_csv(Path(scenario_directory) / f"highest_model_monthly_prediction_weather_by_facility_KDBall_ANC_downscaled_CIL_{scenario}.csv", index=False)


ssp245
Models of interest ['HadGEM3-GC31-LL', 'GFDL-CM4', 'MIROC-ES2L']
date
Akasale Pvt Clinic
Area 30 Police Clinic
Balaka District Hospital
Bangwe Health Centre
Benga Health Centre
Bilira Health Centre
Bolero Rural Hospital
Bondo Health Centre
Bula Health Centre
Bulala Health Centre
Bvumbwe Health Centre
Bwanje Health Centre
Chadza Health Centre
Chakhaza Health Centre
Chamama Health Centre
Chambe Health Centre
Chang'ambika Health Centre
Chapananga Health Centre
Chifunga Health Centre
Chikande Health Centre
Chikole Dispensary
Chikuse Health Centre
Chikwawa District Hospital
Chikweo Health Centre
Chikwina Health Centre
Chileka Health Centre
Chileka Sda Health Centre
Chilonga Health Centre
Chilumba Rural Hospital
Chimatiro Health Centre
Chimbalanga Health Centre
Chimoto Health Centre
Chingale Health Centre
Chingazi Health Centre
Chintheche Rural Hospital
Chinyama Health Centre
Chioshya Health Centre
Chipho Health Centre
Chipoka Health Centre
Chiradzulu District Hospital
Chiringa Health

/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:110: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_lowest_monthly[reporting_facility] = cumulative_sum_monthly
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:111: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_lowest_window[reporting_facility] = cumulative_sum_window
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:113: PerformanceWarning: DataFrame is highly fragmented

Kasina Health Centre
Kasinje Health Centre
Kasoba Health Centre
Kasungu District Hospital
Katchale Health Centre
Katimbila Health Centre
Katowo Rural Hospital
Katsekera Health Centre
Katuli Health Centre
Kaundu Health Centre
Kavuzi Health Centre
Kawamba Health Centre
Kaweche Health Centre
Kawinga Dispensary
Kayembe Health Centre
Khanda Health Centre


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:110: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_lowest_monthly[reporting_facility] = cumulative_sum_monthly
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:111: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_lowest_window[reporting_facility] = cumulative_sum_window
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:113: PerformanceWarning: DataFrame is highly fragmented

Khola Health Centre
Khondowe Health Centre
Khosolo Health Centre
Khuwi Health Centre
Khuyukuyu Health Centre
Kochilira Rural Hospital
Kukalanga Dispensary
Kunenekude Health Centre
Kwitanda Health Centre
Lambulira Health Centre
Lemwe Health Centre
Lilongwe City Assembly Chinsapo
Limbe Health Centre
Lirangwe Health Centre
Liuzi Health Centre
Livwezi Health Centre


...

Ntcheu District Hospital
Ntchisi District Hospital
Nthalire Health Centre
Nthungwa Health Centre
Nyambi Health Centre
Nyamithuthu Health Centre
Nyungwe Health Centre
Nzama Health Centre
Pensulo Health Centre
Phalombe Health Centre
Phirilongwe Health Centre
Phokera Health Centre
Raiply Clinic
Rumphi District Hospital
Salima District Hospital
Sankhulani Health Centre


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:113: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_median_monthly[reporting_facility] = cumulative_sum_monthly
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_median_window[reporting_facility] = cumulative_sum_window
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:116: PerformanceWarning: DataFrame is highly fragmented

Sharpe Valley Health Centre
Somba Health Centre
Sorgin Health Centre
South Lunzu Health Centre
Tembwe Health Centre
Thekerani Rural Hospital
Thembe Health Centre
Thomas Health Centre
Thonje Health Centre
Thumbwe Health Centre
Thunduwike Health Centre
Thyolo District Hospital
Usisya Health Centre
Utale 1 Health Centre
Utale 2 Health Centre
Wiliro Health Centre


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:116: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_highest_monthly[reporting_facility] = cumulative_sum_monthly
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weather_df_highest_window[reporting_facility] = cumulative_sum_window
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_42435/3240928208.py:110: PerformanceWarning: DataFrame is highly fragment

Wimbe Health Centre
Zomba Central Prison Clinic
Zomba Mental Hospital
